In [1]:
#INSTALLS
!pip install yfinance==0.2.41
!pip install crewai==0.28.8
!pip install "crewai[tools]"
!pip install langchain==0.1.20
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.0.38
!pip install duckduckgo-search==5.3.0

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 1.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 KB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 9.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 9.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 9.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 KB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 KB 1.3 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.3 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproj

In [22]:
#IMPORTS
import json 
import os
from datetime import datetime

import yfinance as yf

from crewai import Agent, Crew, Process, Task

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from IPython.display import Markdown

In [23]:
# YAHOO FINANCE 
def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2024-01-01", end="2024-10-09")
    return stock

yahoo_finance_tool = Tool(
    name="Yahoo Finance Tool",
    description="Fetches this year's {ticket} stock prices for a specific company from the Yahoo Finance API",
    func=lambda ticket: fetch_stock_price(ticket),
)

In [24]:
#OPENAI LLM - GPT
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [25]:
stockPriceAnalyst = Agent(
    role= "Senior Stock Price Analyst",
    goal="Find the {ticket} stock price and analyses trends",
    backstory="""You have a lot of experience analyzing the price of a specific stock 
    and making predictions about its future price.""",
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[yahoo_finance_tool],
)

In [26]:
getStockPrice = Task(
    description="Analyze {ticket} stock price history and create uṕ, down or sideways trend analysis",
    expected_output="""Specify the current trend stock price - up, down or sideways. 
    eg. stock= 'APPL, price UP'""",
    agent=stockPriceAnalyst,
)

In [27]:
# DUCKDUCKGO
search_tool = DuckDuckGoSearchResults(backend="news", num_results=10)

In [28]:
newsAnalyst = Agent(
    role= "Stock News Analyst",
    goal="""Create a brief summary of market news related to the stock {ticket} company. 
    Specify the current trend - up, down or sideways with the news context. 
    For each requested stock asset, specify a number between 0 and 100, 
    where 0 is extreme fear and 100 is extreme positivity.""",
    backstory="""You have a lot of experience in analyzing market trends and news and have been following assets for over 10 years.

    You are also a master-level analyst in traditional markets and have in-depth knowledge of human psychology.

    You understand the news, its headlines and information, but you look at it with a healthy dose of skepticism. 
    You also consider the source of the news articles.""",
    verbose=True,
    llm=llm,
    max_iter=10,
    memory=True,
    tools=[search_tool],
    allow_delegation=False,
)

In [29]:
get_news = Task(
    description=f"""Take the stock and always include BTC in it (if not requested).
    Use the search tool to search for each one individually. 
    The current date is {datetime.now()}.
    Compose the results into a useful report""",
    expected_output="""A summary of the overall market and a one-sentence summary for each asset requested. 
    Include a fear/positivity score for each asset based on the news. Use format:
    <STOCK ASSET>
    <NEWS-BASED SUMMARY>
    <TREND FORECAST>
    <FEAR/POSITIVITY SCORE>""",
    agent=newsAnalyst,
)

In [30]:
stockAnalystWriter = Agent(
    role="Senior Stock Analyts Writer",
    goal="""Analyze price trends and news and write an insightful, attractive and informative 3-paragraph 
    newsletter based on the stock report and price trend.""",
    backstory="""You are widely accepted as the best stock analyst in the market. You understand complex concepts 
    and create compelling stories and narratives that resonate with broader audiences. 
    You understand macro factors and combine multiple theories - e.g. cycle theory and fundamental analysis. 
    You are capable of having multiple opinions when analyzing anything.""",
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    allow_delegation=True,
)

In [31]:
writeAnalyses = Task(
    description="""Use stock price trend and stock news report to create analysis and write company newsletter about the {ticket} company, 
    this is brief and highlights the most important points.
    Focus on the stock price trend, news, and fear/positivity score. What are the considerations for the near future?
    Includes previous stock trend analysis and news summary.""",
    expected_output="""An eloquent 3-paragraph newsletter formatted as markdown in an easy-to-read manner. It must contain:
    - 3 bullet executive summary 
    - Introduction - set the big picture and increase interest
    - Main part provides the gist of the analysis, including news summary and fead/positivity scores
    - Summary - important facts and concrete prediction of future trends - up, down or sideways.""",
    agent=stockAnalystWriter,
    context=[getStockPrice, get_news],
)

In [32]:
crew = Crew(
    agents=[stockPriceAnalyst, newsAnalyst, stockAnalystWriter],
    tasks=[getStockPrice, get_news, writeAnalyses],
    verbose=2,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15
)

2024-09-20 00:29:12,932 - 129483320606720 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [33]:
results = crew.kickoff(inputs={"ticket": "AAPL"})

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Analyze AAPL stock price history and create uṕ, down or sideways trend analysis


> Entering new CrewAgentExecutor chain...
I need to gather information from my team to properly analyze the AAPL stock price history and determine the current trend.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior Stock Price Analyst", "task": "Analyze AAPL stock price history and identify any patterns", "context": "Provide a detailed analysis of AAPL stock price history including key trends and patterns"}

> Entering new CrewAgentExecutor chain...
I need to use the Yahoo Finance Tool to fetch this year's AAPL stock prices and analyze the data to identify trends and patterns.

Action: Yahoo Finance Tool
Action Input: {"ticket": "AAPL"}

[*********************100%%**********************]  1 of 1 completed


 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2024-01-02  187.149994  188.440002  183.889999  185.639999  184.938217   
2024-01-03  184.220001  185.880005  183.429993  184.250000  183.553467   
2024-01-04  182.149994  183.089996  180.880005  181.910004  181.222336   
2024-01-05  181.990005  182.759995  180.169998  181.179993  180.495071   
2024-01-08  182.089996  185.600006  181.500000  185.559998  184.858521   
...                ...         ...         ...         ...         ...   
2024-09-13  223.580002  224.039993  221.910004  222.500000  222.500000   
2024-09-16  216.539993  217.220001  213.919998  216.320007  216.320007   
2024-09-17  215.750000  216.899994  214.500000  216.789993  216.789993   
2024-09-18  217.550003  222.710007  217.539993  220.690002  220.690002   
2024-09-19  224.990005  229.820007  224.630005  228.869995  228.869995   

              Volume  
Date       

[*********************100%%**********************]  1 of 1 completed

I should use the Yahoo Finance Tool to fetch this year's AAPL stock prices and analyze trends to provide insights for the report.

Action: Yahoo Finance Tool
Action Input: {"ticker": "AAPL"} 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2024-01-02  187.149994  188.440002  183.889999  185.639999  184.938217   
2024-01-03  184.220001  185.880005  183.429993  184.250000  183.553467   
2024-01-04  182.149994  183.089996  180.880005  181.910004  181.222336   
2024-01-05  181.990005  182.759995  180.169998  181.179993  180.495071   
2024-01-08  182.089996  185.600006  181.500000  185.559998  184.858521   
...                ...         ...         ...         ...         ...   
2024-09-13  223.580002  224.039993  221.910004  222.500000  222.500000   
2024-09-16  216.539993  217.220001  213.919998  216.320007  216.320007   
2024-09-17  215.750000  216.899994  214.500000  216.789993  216.789

Final Answer: 
Based on the analysis of this year's AAPL stock prices, we can see that the stock has shown a general upward trend, with fluctuations along the way. The stock opened the year at $187.15 and closed at $228.87 on September 19th. The highest price reached was $229.82, while the lowest was $180.17. 

It is important to note that the stock experienced significant volatility throughout the year, with trading volume varying from around 58 million to over 82 million shares on different days. 

Overall, the stock has shown growth potential, but investors should be aware of the fluctuations and keep a close eye on market trends to make informed decisions.

> Finished chain.
 

Based on the analysis of this year's AAPL stock prices, we can see that the stock has shown a general upward trend, with fluctuations along the way. The stock opened the year at $187.15 and closed at $228.87 on September 19th. The highest price reached was $229.82, while the lowest was $180.17. 

It is impor

In [34]:
list(results.keys())

['final_output', 'tasks_outputs']

In [36]:
results["final_output"]

"# AAPL Company Newsletter\n\n## Executive Summary:\n- AAPL stock has shown a general upward trend throughout the year, with fluctuations in prices.\n- Recent news highlights potential areas of growth beyond iPhone sales and discusses challenges in the Chinese market.\n- Analysts recommend monitoring developments surrounding Apple stock closely.\n\n## Introduction:\nWelcome to the AAPL Company Newsletter! In this edition, we will dive into the analysis of AAPL stock, recent news highlights, and predictions for the near future. Let's explore the exciting world of Apple's stock market performance.\n\n## Main Part:\nAfter analyzing the AAPL stock price history for this year, it is evident that the stock has experienced fluctuations but maintained an overall upward trend. The stock opened at $187.15 and closed at $228.87 on September 19th, with the highest price reaching $229.82 and the lowest at $180.17. Trading volume has varied significantly throughout the year, ranging from 58 million 

In [37]:
len(results["tasks_outputs"])

3

In [38]:
Markdown(results["final_output"])

# AAPL Company Newsletter

## Executive Summary:
- AAPL stock has shown a general upward trend throughout the year, with fluctuations in prices.
- Recent news highlights potential areas of growth beyond iPhone sales and discusses challenges in the Chinese market.
- Analysts recommend monitoring developments surrounding Apple stock closely.

## Introduction:
Welcome to the AAPL Company Newsletter! In this edition, we will dive into the analysis of AAPL stock, recent news highlights, and predictions for the near future. Let's explore the exciting world of Apple's stock market performance.

## Main Part:
After analyzing the AAPL stock price history for this year, it is evident that the stock has experienced fluctuations but maintained an overall upward trend. The stock opened at $187.15 and closed at $228.87 on September 19th, with the highest price reaching $229.82 and the lowest at $180.17. Trading volume has varied significantly throughout the year, ranging from 58 million to over 82 million shares on different days. Recent news indicates caution with potential downside risks, but also highlights areas of growth beyond iPhone sales, such as advertising revenue and software services. Analysts believe that monitoring developments surrounding Apple stock will be crucial in the near future.

## Summary:
In conclusion, AAPL stock has shown resilience amidst market fluctuations, with a general upward trend. Recent news highlights both challenges and opportunities for Apple, urging investors to stay informed and vigilant. As we move forward, it will be essential to keep a close eye on how Apple navigates the evolving market landscape to make informed investment decisions.